# Overview

## Setting Up Your Environment

In preparation for this tutorial we will need to set up the proper environment: 

* configure Google Colab
* install custom libraries
* download data
* Python imports

### Google Colab

If you are running this tutorial in Google Colab, the first important thing to do is switch to a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

Next we need to select and/or upgrade a few specific library versions. We will do this using the following Jupyter magic commands:

In [ ]:
# --- Select Tensorflow 2.0, upgrade pyyaml (only in Google Colab)
%tensorflow_version 2.x
%pip install -U pyyaml

### Custom libraries

This tutorial will use several custom, external Python libraries to facilitate low-level data management, data visualization and other useful tools optimized for machine learning and other data science projects in healthcare. More information and additional tutorials may be found at the following GitHub repositories: 

* https://github.com/peterchang77/dl_train
* https://github.com/peterchang77/dl_utils

In [ ]:
# --- Install dl_utils and dl_train library
!wget -O setenv.py https://raw.githubusercontent.com/peterchang77/dl_utils/master/setenv.py
from setenv import prepare_env
prepare_env()

### Download data

Next we will download and prepare data for this tutorial:

In [ ]:
from dl_train import datasets
datasets.download(name='brats')

### Python imports 

The following modules will be used in this tutorial:

In [ ]:
import glob, os
import numpy as np

from tensorflow.keras import Input, Model, layers, models, losses, optimizers
from tensorflow import math

from dl_utils.display import imshow

# Data

The data you have downloaded above contains preprocessed images and labels for this tutorial. To access the data, we will prepare Python generators (`gen_train` and `gen_valid`) using the custom `datasets` module. In addition, a custom `client` object will be created that will facilitate interaction with data.

The `datasets.prepare(...)` method accepts a `configs` variable that defines:

* `size`: training batch size
* `fold`: fold to use for validation
* `sampling`: stratified sampling strategy

In this tutorial, we will use the following settings:

In [ ]:
# --- Prepare Python generators
gen_train, gen_valid, client = datasets.prepare(name='brats', configs={
    'batch': {
        'size': 16,          # ==> Use a batch size of 16
        'fold': 0,           # ==> Use the first fold (=0) for validation
        'sampling': {        # ==> Use a 50/50% stratified sampling of foreground and background
            'fg': 0.5,
            'bg': 0.5}}
})

The returned Python generators yield a tuple `(xs, ys)` that conform to the Tensorflow / Keras 2.0 API for training input(s) and output(s). Let us take a closer look here:

In [ ]:
# --- Yield the next batch
xs, ys = next(gen_train)

# --- Inspect xs and ys dictionaries
print(xs.keys())
print(ys.keys())

# --- Inspect the `t2` array
print(xs['t2'].shape)

### Visualization

Let us now view the underlying voxel data using the `imshow()` method available in the custom `dl_utils.display` module. This useful function can be used to directly visualize any 2D slice of data (first argument), as well as overlay any mask if optionally provided (second argument). 

Example usage as follows:

In [ ]:
imshow(xs['flair'], ys['lbl'], figsize=(12, 12))

### Tensorflow / Keras Input() tensors

Training input(s) are passed into a Tensorflow / Keras model via specific `Input()` objects, defined via `tf.keras.Input(...)`. For each input, the corresponding tensor `shape` and `dtype` should be defined. A convenience function as part of the custom `client` class can be used to generate corresponding `Input()` objects for all the arrays in `xs`:

In [ ]:
# --- Create Input() objects
inputs = client.get_inputs(Input)

for key, i in inputs.items():
    print('{}: {}'.format(key, i))

# Model

In this exercise, we will create a custom variant of the standard contracting-expanding netowrk topology, popularly referred to as a U-Net architecture. We will define the algorithm completely here in the next several code cells using the functional API of Tensorflow/Keras. For a more general overview of basic Tensorflow/Keras usage, see the following tutorial links (remote/local). 

## Creating a Convolutional Block

To help facilitate concise and readable code, we will create template Python lambda functions to succintly define convolutional blocks, defined as the following series of consecutive operations:

* convolution (or convolutional-transpose)
* batch normalization
* activation function (ReLU, leaky ReLU, etc)

In [ ]:
# --- Define convolution parameters
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same',
    'kernel_initializer': 'he_normal'}

# --- Define block components
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2)))) 

Now, we are ready to define the full model.

In [ ]:
def create_model(inputs):
    """
    Method to create simple U-Net architecture

    """
    # --- Concatenate modalities
    combined = layers.concatenate([
        inputs['t2'],
        inputs['flair'],
        inputs['pre'],
        inputs['post']
    ])

    # --- Define contracting layers
    l1 = conv1(16, combined)
    l2 = conv1(32, conv2(32, l1))
    l3 = conv1(48, conv2(48, l2))
    l4 = conv1(64, conv2(64, l3))
    l5 = conv1(80, conv2(80, l4))

    # --- Define expanding layers
    l6 = tran2(64, l5)
    l7 = tran2(48, conv1(64, l6 + l4))
    l8 = tran2(32, conv1(48, l7 + l3))
    l9 = tran2(16, conv1(32, l8 + l2))

    logits = {}
    logits['lbl'] = layers.Conv3D(filters=2, name='lbl', **kwargs)(conv1(16, l1 + l9))

    return Model(inputs=inputs, outputs=logits)

In [ ]:
# --- Create model and show summary
model = create_model(inputs)
model.summary(line_length=120)

## Preparing the Model

Next, we must compile the model with the requisite objects that define training dynamics (e.g. how the algorithm with learn). This will include classes that encapsulate the model `optimizer`, `loss` and `metrics` for evaluating algorithm performance. For more information about how these strategies are determined and defined, see the following tutorial links (remote/local).

In [ ]:
# --- Define optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Define loss
loss = losses.SparseCategoricalCrossentropy(from_logits=True)

### Custom Metric

In addition the standard evaluation metric of `accuracy` (% of pixels or voxels that are predicted correctly), we will also keep track of a common metric to evaluate spatial overlap of masks: the Dice score. To do so, we need to create a custom metric function. For more information about creating custom metrics (and losses) in Tensorflow/Keras, see the following tutorial links (remote/local).

In [ ]:
def dsc(epsilon=1):

    def dice(y_true, y_pred):

        true = y_true[..., 0] == 1
        pred = y_pred[..., 1] > y_pred[..., 0] 

        A = math.count_nonzero(true & pred) * 2
        B = math.count_nonzero(true) + math.count_nonzero(pred) + epsilon

        return A / B

    return dice

### Compile

At last we are ready to compile the model. This is done simply with a call using the `model.compile()` method:

In [ ]:
model.compile(
    optimizer=optimizer,
    loss={'lbl': loss},
    metrics={'lbl': dsc()})

# Training

One of the primary advantages to the Tensorflow/Keras API is that by following the above "recipe" to customize, instantiate and compile a `model` object, several very easy-to-use interfaces are available for algorithm training. In this tutorial, we will use data prepared from Python generators (`gen_train` and `gen_valid` as above) to train the model using the `model.fit_generator()` method. Usage is shown as follows using a single line of code:

In [ ]:
# --- Train the model
model.fit_generator(
    generator=gen_train,
    steps_per_epoch=500,
    epochs=4,
    validation_data=gen_valid,
    validation_steps=100)

# Prediction

How did we do? The validation performance metrics (accuracy, Dice score) give us a reasonable benchmark, but the most important thing to do at the end of the day is to visually check some examples for yourself. Let us pass some validation data manually into the model using the `model.predict()` method and see some results:

In [ ]:
# --- Load data and preproces
arrays = client.get(split='valid', cohort='fg')

# --- Run prediction
pred = model.predict({k: np.expand_dims(v, axis=0) for k, v in arrays['xs'].items()})
mask = np.argmax(pred[0], axis=-1)

# --- Show prediction
imshow(arrays['xs']['flair'], mask)

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `model.save()` and `models.load_model()` methods. Note that any custom losses and/or metrics will need to be provided via a dictionary.

In [ ]:
# --- Serialize a model
os.makedirs('./models', exist_ok=True)
model.save('./models/unet.hdf5')

In [ ]:
# --- Load a serialized model
model = models.load_model('./models/unet.hdf5', custom_objects={'dice': dsc()})